<table style="width: 100%;">
    <tr>
        <td><a href="https://ieb-chile.cl/en/" target="_blank"><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/IEB.png" style="height: 100px;"></a</td>
        <td></td>
        <td><img src="https://raw.githubusercontent.com/IEB-BIODATA/pydwca-examples/main/images/logo/Biodata.png" style="height: 100px;"></td>
    </tr>
</table>

# Catalogue of Vascular Plats of Chile

A usage example of the `PyDwCA` library, writing a DwC-A from a database of biodiversity data.

> _Note_: This notebook works directly with the database of the [Digital Herbarium of Chile](https://herbariodigital.cl/), therefore, this code is not replicable, but it is presented as an example for the generation of a DwC-A file using previous populated data.

In [1]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

In [2]:
if load_dotenv():
    print("Loaded")
else:
    print("Something went wrong")

Loaded


## Populating EML

Here we generate an EML metadata file with the needed information.

In [3]:
from dwca import DarwinCoreArchive
from eml.types import ResponsibleParty, OrganizationName, PositionName, I18nString
from xml_common.utils import Language

In [4]:
catalogue = DarwinCoreArchive("Catalogue of Vascular Plants of Chile")
catalogue.generate_eml("eml.xml")
catalogue.metadata.initialize_resource(
    "Catalogue of Vascular Plants of Chile",
    ResponsibleParty(
        organization_name=OrganizationName(I18nString("Departamento de Botánica, Facultad de Ciencias Naturales y Oceanográficas, Universidad de Concepción", Language.ESP)),
    ),
    contact=[ResponsibleParty(
        position_name=PositionName("Informatic Researcher")
    )]
)

## Standardizing Data

Processing the data to meet the Darwin Core standard.

In [5]:
from dwca.classes import Taxon
from dwca.terms import TaxonID, ParentNameUsageID, AcceptedNameUsageID, OriginalNameUsageID, ScientificNameID, \
    DWCDataset, TaxonomicStatus, TaxonRank, TaxonRemarks, ScientificName, Kingdom, Phylum, DWCClass, Order, Family, Genus, \
    GenericName, InfragenericEpithet, SpecificEpithet, InfraspecificEpithet, ScientificNameAuthorship, \
    NameAccordingTo, NamePublishedIn, NomenclaturalCode, NomenclaturalStatus

In [6]:
catalogue_core = Taxon(
    0, "taxon.xml",
    [
        TaxonID(0), ParentNameUsageID(1), AcceptedNameUsageID(2),
        OriginalNameUsageID(3), ScientificNameID(4), DWCDataset(5),
        TaxonomicStatus(6), TaxonRank(7), TaxonRemarks(8), ScientificName(9),
        Phylum(10), Kingdom(11), DWCClass(12), Order(13), Family(14), Genus(15),
        GenericName(16), InfragenericEpithet(17), SpecificEpithet(18),
        InfraspecificEpithet(19), ScientificNameAuthorship(20),
        NameAccordingTo(21), NamePublishedIn(22),
        NomenclaturalCode(23), NomenclaturalStatus(24)
    ]
)

Connect to the database

In [7]:
connection = psycopg2.connect(
    dbname=os.getenv("HERBARIUM_DB"),
    host=os.getenv("HERBARIUM_HOST"),
    user=os.getenv("HERBARIUM_USER"),
    password=os.getenv("HERBARIUM_PASS"),
    port=os.getenv("HERBARIUM_PORT"),
)

Getting the highest ranks: kingdom, division (phylum), class, order, family and genus.

In [8]:
taxon_ranks = [
    ('kingdom', 'kingdom'),
    ('division', 'phylum'),
    ('classname', 'class'),
    ('order', 'order'),
    ('family', 'family'),
    ('genus', 'genus'),
]

In [9]:
connection.rollback()
base_sql = """
SELECT catalog_{table}.taxon_id "taxonID",
    {parent} "parentNameUsageID", catalog_{table}.taxon_id "acceptedNameUsageID", 
    catalog_{table}.taxon_id "originalNameUsageID", catalog_{table}.taxon_id "scientificNameID",
    'Catalogue of Vascular Plants of Chile' "datasetID",
    'accepted' "taxonomicStatus", '{rank}' "taxonRank",
    catalog_{table}.name "scientificName",
    {kingdom} "kingdom", {phylum} "phylum", {class} "class",
    {order} "order", {family} "family",
    {genus} "genus", 'ICBN' "nomenclaturalCode"
FROM {from_table}"""
rank_query = {
    "parent": "NULL",
    "rank": "NULL",
    "kingdom": "NULL",
    "phylum": "NULL",
    "class": "NULL",
    "order": "NULL",
    "family": "NULL",
    "genus": "NULL",
    "table": "NULL",
    "from_table": ""
}
previous_table = "NULL"
results = list()
join_statement = ""
with connection.cursor() as cursor:
    for table, rank in taxon_ranks:
        rank_query["table"] = table
        rank_query[rank] = f"catalog_{table}.name"
        rank_query["rank"] = rank.lower()
        if previous_table != "NULL":
            join_statement = f"""
    JOIN catalog_{previous_table} ON catalog_{table}.{previous_table}_id = catalog_{previous_table}.id """.format(
                previous_table=previous_table, table=table
            ) + join_statement
        rank_query["from_table"] = f"catalog_{table}\n\t{join_statement}"
        cursor.execute(base_sql.format(**rank_query))
        rank_query["parent"] = "catalog_" + table + ".taxon_id"
        results.append(
            pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
        )
        previous_table = table
        
result = pd.concat(results)
result

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,kingdom,phylum,class,order,family,genus,nomenclaturalCode
0,https://catalogoplantas.udec.cl/taxa/1,None,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/1,Catalogue of Vascular Plants of Chile,accepted,kingdom,Plantae,Plantae,None,None,None,None,None,ICBN
0,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,https://catalogoplantas.udec.cl/taxa/2,Catalogue of Vascular Plants of Chile,accepted,phylum,Magnoliophyta,Plantae,Magnoliophyta,None,None,None,None,ICBN
1,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/3,Catalogue of Vascular Plants of Chile,accepted,phylum,Pinophyta,Plantae,Pinophyta,None,None,None,None,ICBN
2,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/1,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,https://catalogoplantas.udec.cl/taxa/4,Catalogue of Vascular Plants of Chile,accepted,phylum,Pteridophyta,Plantae,Pteridophyta,None,None,None,None,ICBN
0,https://catalogoplantas.udec.cl/taxa/5,https://catalogoplantas.udec.cl/taxa/3,https://catalogoplantas.udec.cl/taxa/5,https://catalogoplantas.udec.cl/taxa/5,https://catalogoplantas.udec.cl/taxa/5,Catalogue of Vascular Plants of Chile,accepted,class,Gnetopsida,Plantae,Pinophyta,Gnetopsida,None,None,None,ICBN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,https://catalogoplantas.udec.cl/taxa/1406,https://catalogoplantas.udec.cl/taxa/239,https://catalogoplantas.udec.cl/taxa/1406,https://catalogoplantas.udec.cl/taxa/1406,https://catalogoplantas.udec.cl/taxa/1406,Catalogue of Vascular Plants of Chile,accepted,genus,Zephyra,Plantae,Magnoliophyta,Liliopsida,Asparagales,Tecophilaeaceae,Zephyra,ICBN
1153,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/73,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/1407,Catalogue of Vascular Plants of Chile,accepted,genus,Zephyranthes,Plantae,Magnoliophyta,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,ICBN
1154,https://catalogoplantas.udec.cl/taxa/1408,https://catalogoplantas.udec.cl/taxa/252,https://catalogoplantas.udec.cl/taxa/1408,https://catalogoplantas.udec.cl/taxa/1408,https://catalogoplantas.udec.cl/taxa/1408,Catalogue of Vascular Plants of Chile,accepted,genus,Zostera,Plantae,Magnoliophyta,Liliopsida,Alismatales,Zosteraceae,Zostera,ICBN
1155,https://catalogoplantas.udec.cl/taxa/20091,https://catalogoplantas.udec.cl/taxa/203,https://catalogoplantas.udec.cl/taxa/20091,https://catalogoplantas.udec.cl/taxa/20091,https://catalogoplantas.udec.cl/taxa/20091,Catalogue of Vascular Plants of Chile,accepted,genus,Microlaena,Plantae,Magnoliophyta,Liliopsida,Poales,Poaceae,Microlaena,ICBN


Getting species and synonymy.

In [10]:
connection.rollback()
with connection.cursor() as cursor:
    cursor.execute("""
SELECT catalog_species.taxon_id "taxonID",
    'https://catalogoplantas.udec.cl/taxa/' || catalog_species.parent_taxon_id "parentNameUsageID",
    catalog_species.taxon_id "acceptedNameUsageID", 
    catalog_species.taxon_id "originalNameUsageID", catalog_species.taxon_id "scientificNameID",
    'Catalogue of Vascular Plants of Chile' "datasetID",
    'accepted' "taxonomicStatus", tr.name_en "taxonRank",
    CASE
        WHEN catalog_species.taxon_rank_id = 1 THEN 
            CASE
                WHEN catalog_species.scientific_name_authorship IS NOT NULL THEN catalog_genus.name || ' ' || catalog_species.specific_epithet || ' ' || catalog_species.scientific_name_authorship
                ELSE catalog_genus.name || ' ' || catalog_species.specific_epithet
            END
        WHEN catalog_species.taxon_rank_id = 2 THEN 
            CASE
                WHEN catalog_species.ssp_authorship IS NOT NULL THEN catalog_genus.name || ' ' || catalog_species.specific_epithet || ' ssp. ' || catalog_species.subspecies || ' ' || catalog_species.ssp_authorship
                ELSE catalog_genus.name || ' ' || catalog_species.specific_epithet || ' ssp. ' || catalog_species.subspecies
            END
        WHEN catalog_species.taxon_rank_id = 3 THEN 
            CASE
                WHEN catalog_species.variety_authorship IS NOT NULL THEN catalog_genus.name || ' ' || catalog_species.specific_epithet || ' var. ' || catalog_species.variety || ' ' || catalog_species.variety_authorship
                ELSE catalog_genus.name || ' ' || catalog_species.specific_epithet || ' var. ' || catalog_species.variety
            END
        WHEN catalog_species.taxon_rank_id = 4 THEN 
            CASE
                WHEN catalog_species.form_authorship IS NOT NULL THEN catalog_genus.name || ' ' || catalog_species.specific_epithet || ' fma. ' || catalog_species.form || ' ' || catalog_species.form_authorship
                ELSE catalog_genus.name || ' ' || catalog_species.specific_epithet || ' fma. ' || catalog_species.form
            END
    END AS "scientificName",
    catalog_kingdom.name "kingdom", catalog_division.name "phylum", catalog_classname.name "class",
    catalog_order.name "order", catalog_family.name "family",
    catalog_genus.name "genus",
    catalog_genus.name "genericName",
    NULL "infragenericEpithet",
    catalog_species.specific_epithet "specificEpithet",
    CASE
        WHEN catalog_species.taxon_rank_id = 1 THEN NULL
        WHEN catalog_species.taxon_rank_id = 2 THEN catalog_species.subspecies
        WHEN catalog_species.taxon_rank_id = 3 THEN catalog_species.variety
        WHEN catalog_species.taxon_rank_id = 4 THEN catalog_species.form
    END AS "infraspecificEpithet",
    CASE
        WHEN catalog_species.taxon_rank_id = 1 THEN catalog_species.scientific_name_authorship
        WHEN catalog_species.taxon_rank_id = 2 THEN catalog_species.ssp_authorship
        WHEN catalog_species.taxon_rank_id = 3 THEN catalog_species.variety_authorship
        WHEN catalog_species.taxon_rank_id = 4 THEN catalog_species.form_authorship
    END AS "scientificNameAuthorship",
    'ICBN' "nomenclaturalCode"
FROM catalog_species
    JOIN catalog_genus ON catalog_species.genus_id = catalog_genus.id
    JOIN catalog_family ON catalog_genus.family_id = catalog_family.id 
    JOIN catalog_order ON catalog_family.order_id = catalog_order.id 
    JOIN catalog_classname ON catalog_order.classname_id = catalog_classname.id 
    JOIN catalog_division ON catalog_classname.division_id = catalog_division.id 
    JOIN catalog_kingdom ON catalog_division.kingdom_id = catalog_kingdom.id
    JOIN catalog_taxonrank tr ON catalog_species.taxon_rank_id = tr.id
    """)
    species_result = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
species_result

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,kingdom,...,class,order,family,genus,genericName,infragenericEpithet,specificEpithet,infraspecificEpithet,scientificNameAuthorship,nomenclaturalCode
0,https://catalogoplantas.udec.cl/taxa/3897,https://catalogoplantas.udec.cl/taxa/20124,https://catalogoplantas.udec.cl/taxa/3897,https://catalogoplantas.udec.cl/taxa/3897,https://catalogoplantas.udec.cl/taxa/3897,Catalogue of Vascular Plants of Chile,accepted,subspecies,Gentiana gayi ssp. gayi,Plantae,...,Magnoliopsida,Gentianales,Gentianaceae,Gentiana,Gentiana,None,gayi,gayi,None,ICBN
1,https://catalogoplantas.udec.cl/taxa/2133,https://catalogoplantas.udec.cl/taxa/2129,https://catalogoplantas.udec.cl/taxa/2133,https://catalogoplantas.udec.cl/taxa/2133,https://catalogoplantas.udec.cl/taxa/2133,Catalogue of Vascular Plants of Chile,accepted,subspecies,Baccharis tola ssp. tola,Plantae,...,Magnoliopsida,Asterales,Asteraceae,Baccharis,Baccharis,None,tola,tola,None,ICBN
2,https://catalogoplantas.udec.cl/taxa/3898,https://catalogoplantas.udec.cl/taxa/20124,https://catalogoplantas.udec.cl/taxa/3898,https://catalogoplantas.udec.cl/taxa/3898,https://catalogoplantas.udec.cl/taxa/3898,Catalogue of Vascular Plants of Chile,accepted,subspecies,Gentiana gayi ssp. magellanica (Kuns.) S. Pfan...,Plantae,...,Magnoliopsida,Gentianales,Gentianaceae,Gentiana,Gentiana,None,gayi,magellanica,(Kuns.) S. Pfanzelt & K.B. Hagen,ICBN
3,https://catalogoplantas.udec.cl/taxa/4219,https://catalogoplantas.udec.cl/taxa/20127,https://catalogoplantas.udec.cl/taxa/4219,https://catalogoplantas.udec.cl/taxa/4219,https://catalogoplantas.udec.cl/taxa/4219,Catalogue of Vascular Plants of Chile,accepted,subspecies,Hymenophyllum ferrugineum ssp. ferrugineum,Plantae,...,Polypodiopsida,Hymenophyllales,Hymenophyllaceae,Hymenophyllum,Hymenophyllum,None,ferrugineum,ferrugineum,None,ICBN
4,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20129,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20113,Catalogue of Vascular Plants of Chile,accepted,subspecies,Koeleria spicata ssp. spicata,Plantae,...,Liliopsida,Poales,Poaceae,Koeleria,Koeleria,None,spicata,spicata,None,ICBN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5874,https://catalogoplantas.udec.cl/taxa/7255,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7255,https://catalogoplantas.udec.cl/taxa/7255,https://catalogoplantas.udec.cl/taxa/7255,Catalogue of Vascular Plants of Chile,accepted,species,Zephyranthes sarae J.M. Watson & A.R. Flores,Plantae,...,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,Zephyranthes,None,sarae,None,J.M. Watson & A.R. Flores,ICBN
5875,https://catalogoplantas.udec.cl/taxa/7256,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7256,https://catalogoplantas.udec.cl/taxa/7256,https://catalogoplantas.udec.cl/taxa/7256,Catalogue of Vascular Plants of Chile,accepted,species,Zephyranthes splendens (Renjifo) Nic. García,Plantae,...,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,Zephyranthes,None,splendens,None,(Renjifo) Nic. García,ICBN
5876,https://catalogoplantas.udec.cl/taxa/7257,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7257,https://catalogoplantas.udec.cl/taxa/7257,https://catalogoplantas.udec.cl/taxa/7257,Catalogue of Vascular Plants of Chile,accepted,species,Zephyranthes tenuiflora (Phil.) Nic.García,Plantae,...,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,Zephyranthes,None,tenuiflora,None,(Phil.) Nic.García,ICBN
5877,https://catalogoplantas.udec.cl/taxa/7258,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/7258,https://catalogoplantas.udec.cl/taxa/7258,https://catalogoplantas.udec.cl/taxa/7258,Catalogue of Vascular Plants of Chile,accept

In [11]:
connection.rollback()
with connection.cursor() as cursor:
    cursor.execute("""
SELECT catalog_synonymy.taxon_id "taxonID",
    'https://catalogoplantas.udec.cl/taxa/' || catalog_species.parent_taxon_id "parentNameUsageID",
    catalog_species.taxon_id "acceptedNameUsageID", 
    catalog_species.taxon_id "originalNameUsageID", catalog_synonymy.taxon_id "scientificNameID",
    'Catalogue of Vascular Plants of Chile' "datasetID",
    'synonym' "taxonomicStatus", tr.name_en "taxonRank",
    CASE
        WHEN catalog_synonymy.taxon_rank_id = 1 THEN 
            CASE
                WHEN catalog_synonymy.scientific_name_authorship IS NOT NULL THEN catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' ' || catalog_synonymy.scientific_name_authorship
                ELSE catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet
            END
        WHEN catalog_synonymy.taxon_rank_id = 2 THEN 
            CASE
                WHEN catalog_synonymy.ssp_authorship IS NOT NULL THEN catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' ssp. ' || catalog_synonymy.subspecies || ' ' || catalog_synonymy.ssp_authorship
                ELSE catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' ssp. ' || catalog_synonymy.subspecies
            END
        WHEN catalog_synonymy.taxon_rank_id = 3 THEN 
            CASE
                WHEN catalog_synonymy.variety_authorship IS NOT NULL THEN catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' var. ' || catalog_synonymy.variety || ' ' || catalog_synonymy.variety_authorship
                ELSE catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' var. ' || catalog_synonymy.variety
            END
        WHEN catalog_synonymy.taxon_rank_id = 4 THEN 
            CASE
                WHEN catalog_synonymy.form_authorship IS NOT NULL THEN catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' fma. ' || catalog_synonymy.form || ' ' || catalog_synonymy.form_authorship
                ELSE catalog_synonymy.genus || ' ' || catalog_synonymy.specific_epithet || ' fma. ' || catalog_synonymy.form
            END
    END AS "scientificName",
    catalog_kingdom.name "kingdom", catalog_division.name "phylum", catalog_classname.name "class",
    catalog_order.name "order", catalog_family.name "family",
    catalog_genus.name "genus",
    catalog_synonymy.genus "genericName",
    NULL "infragenericEpithet",
    catalog_synonymy.specific_epithet "specificEpithet",
    CASE
        WHEN catalog_synonymy.taxon_rank_id = 1 THEN NULL
        WHEN catalog_synonymy.taxon_rank_id = 2 THEN catalog_synonymy.subspecies
        WHEN catalog_synonymy.taxon_rank_id = 3 THEN catalog_synonymy.variety
        WHEN catalog_synonymy.taxon_rank_id = 4 THEN catalog_synonymy.form
    END AS "infraspecificEpithet",
    CASE
        WHEN catalog_synonymy.taxon_rank_id = 1 THEN catalog_synonymy.scientific_name_authorship
        WHEN catalog_synonymy.taxon_rank_id = 2 THEN catalog_synonymy.ssp_authorship
        WHEN catalog_synonymy.taxon_rank_id = 3 THEN catalog_synonymy.variety_authorship
        WHEN catalog_synonymy.taxon_rank_id = 4 THEN catalog_synonymy.form_authorship
    END AS "scientificNameAuthorship",
    'ICBN' "nomenclaturalCode"
FROM catalog_synonymy
    JOIN catalog_species ON catalog_synonymy.species_id = catalog_species.id
    JOIN catalog_genus ON catalog_species.genus_id = catalog_genus.id
    JOIN catalog_family ON catalog_genus.family_id = catalog_family.id 
    JOIN catalog_order ON catalog_family.order_id = catalog_order.id 
    JOIN catalog_classname ON catalog_order.classname_id = catalog_classname.id 
    JOIN catalog_division ON catalog_classname.division_id = catalog_division.id 
    JOIN catalog_kingdom ON catalog_division.kingdom_id = catalog_kingdom.id
    JOIN catalog_taxonrank tr ON catalog_species.taxon_rank_id = tr.id
    """)
    synonymy_result = pd.DataFrame(cursor.fetchall(), columns=[col[0] for col in cursor.description])
synonymy_result

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,kingdom,...,class,order,family,genus,genericName,infragenericEpithet,specificEpithet,infraspecificEpithet,scientificNameAuthorship,nomenclaturalCode
0,https://catalogoplantas.udec.cl/taxa/10569,https://catalogoplantas.udec.cl/taxa/1176,https://catalogoplantas.udec.cl/taxa/6039,https://catalogoplantas.udec.cl/taxa/6039,https://catalogoplantas.udec.cl/taxa/10569,Catalogue of Vascular Plants of Chile,synonym,species,Citharexylum venustum Phil.,Plantae,...,Magnoliopsida,Lamiales,Verbenaceae,Rhaphithamnus,Citharexylum,None,venustum,None,Phil.,ICBN
1,https://catalogoplantas.udec.cl/taxa/7261,https://catalogoplantas.udec.cl/taxa/532,https://catalogoplantas.udec.cl/taxa/2985,https://catalogoplantas.udec.cl/taxa/2985,https://catalogoplantas.udec.cl/taxa/7261,Catalogue of Vascular Plants of Chile,synonym,species,Abutilon bicolor Phil. ex K. Schum.,Plantae,...,Magnoliopsida,Malvales,Malvaceae,Corynabutilon,Abutilon,None,bicolor,None,Phil. ex K. Schum.,ICBN
2,https://catalogoplantas.udec.cl/taxa/7263,https://catalogoplantas.udec.cl/taxa/532,https://catalogoplantas.udec.cl/taxa/2988,https://catalogoplantas.udec.cl/taxa/2988,https://catalogoplantas.udec.cl/taxa/7263,Catalogue of Vascular Plants of Chile,synonym,species,Abutilon garckei Baker f.,Plantae,...,Magnoliopsida,Malvales,Malvaceae,Corynabutilon,Abutilon,None,garckei,None,Baker f.,ICBN
3,https://catalogoplantas.udec.cl/taxa/7264,https://catalogoplantas.udec.cl/taxa/532,https://catalogoplantas.udec.cl/taxa/2987,https://catalogoplantas.udec.cl/taxa/2987,https://catalogoplantas.udec.cl/taxa/7264,Catalogue of Vascular Plants of Chile,synonym,species,Abutilon hirsutum (Phil.) Reiche,Plantae,...,Magnoliopsida,Malvales,Malvaceae,Corynabutilon,Abutilon,None,hirsutum,None,(Phil.) Reiche,ICBN
4,https://catalogoplantas.udec.cl/taxa/7265,https://catalogoplantas.udec.cl/taxa/532,https://catalogoplantas.udec.cl/taxa/2988,https://catalogoplantas.udec.cl/taxa/2988,https://catalogoplantas.udec.cl/taxa/7265,Catalogue of Vascular Plants of Chile,synonym,species,Abutilon ochsenii (Phil.) Phil. ex F. Phil.,Plantae,...,Magnoliopsida,Malvales,Malvaceae,Corynabutilon,Abutilon,None,ochsenii,None,(Phil.) Phil. ex F. Phil.,ICBN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12800,https://catalogoplantas.udec.cl/taxa/20110,https://catalogoplantas.udec.cl/taxa/958,https://catalogoplantas.udec.cl/taxa/4927,https://catalogoplantas.udec.cl/taxa/4927,https://catalogoplantas.udec.cl/taxa/20110,Catalogue of Vascular Plants of Chile,synonym,species,Belloa kunthiana (DC.) Anderb. & S.E. Freire,Plantae,...,Magnoliopsida,Asterales,Asteraceae,Mniodes,Belloa,None,kunthiana,None,(DC.) Anderb. & S.E. Freire,ICBN
12801,https://catalogoplantas.udec.cl/taxa/20111,https://catalogoplantas.udec.cl/taxa/958,https://catalogoplantas.udec.cl/taxa/4927,https://catalogoplantas.udec.cl/taxa/4927,https://catalogoplantas.udec.cl/taxa/20111,Catalogue of Vascular Plants of Chile,synonym,species,Lucilia kunthiana (DC.) Zardini,Plantae,...,Magnoliopsida,Asterales,Asteraceae,Mniodes,Lucilia,None,kunthiana,None,(DC.) Zardini,ICBN
12802,https://catalogoplantas.udec.cl/taxa/20114,https://catalogoplantas.udec.cl/taxa/20129,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20114,Catalogue of Vascular Plants of Chile,synonym,subspecies,Trisetum spicatum ssp. spicatum,Plantae,...,Liliopsida,Poales,Poaceae,Koeleria,Trisetum,None,spicatum,spicatum,None,ICBN
12803,https://catalogoplantas.udec.cl/taxa/18127,https://catalogoplantas.udec.cl/taxa/1246,https://catalogoplantas.udec.cl/taxa/6385,https://catalogoplantas.udec.cl/taxa/6385,https://catalogoplantas.udec.cl/taxa/18127,Catalogue of Vascular Plants of Chile,synonym,species,Senecio leucomallus var. incisus A. Gray,Plantae,...,Magnoliopsida,Asterales,Asteraceae

In [12]:
result = pd.concat([species_result, synonymy_result, result])
result

,taxonID,parentNameUsageID,acceptedNameUsageID,originalNameUsageID,scientificNameID,datasetID,taxonomicStatus,taxonRank,scientificName,kingdom,...,class,order,family,genus,genericName,infragenericEpithet,specificEpithet,infraspecificEpithet,scientificNameAuthorship,nomenclaturalCode
0,https://catalogoplantas.udec.cl/taxa/3897,https://catalogoplantas.udec.cl/taxa/20124,https://catalogoplantas.udec.cl/taxa/3897,https://catalogoplantas.udec.cl/taxa/3897,https://catalogoplantas.udec.cl/taxa/3897,Catalogue of Vascular Plants of Chile,accepted,subspecies,Gentiana gayi ssp. gayi,Plantae,...,Magnoliopsida,Gentianales,Gentianaceae,Gentiana,Gentiana,None,gayi,gayi,None,ICBN
1,https://catalogoplantas.udec.cl/taxa/2133,https://catalogoplantas.udec.cl/taxa/2129,https://catalogoplantas.udec.cl/taxa/2133,https://catalogoplantas.udec.cl/taxa/2133,https://catalogoplantas.udec.cl/taxa/2133,Catalogue of Vascular Plants of Chile,accepted,subspecies,Baccharis tola ssp. tola,Plantae,...,Magnoliopsida,Asterales,Asteraceae,Baccharis,Baccharis,None,tola,tola,None,ICBN
2,https://catalogoplantas.udec.cl/taxa/3898,https://catalogoplantas.udec.cl/taxa/20124,https://catalogoplantas.udec.cl/taxa/3898,https://catalogoplantas.udec.cl/taxa/3898,https://catalogoplantas.udec.cl/taxa/3898,Catalogue of Vascular Plants of Chile,accepted,subspecies,Gentiana gayi ssp. magellanica (Kuns.) S. Pfan...,Plantae,...,Magnoliopsida,Gentianales,Gentianaceae,Gentiana,Gentiana,None,gayi,magellanica,(Kuns.) S. Pfanzelt & K.B. Hagen,ICBN
3,https://catalogoplantas.udec.cl/taxa/4219,https://catalogoplantas.udec.cl/taxa/20127,https://catalogoplantas.udec.cl/taxa/4219,https://catalogoplantas.udec.cl/taxa/4219,https://catalogoplantas.udec.cl/taxa/4219,Catalogue of Vascular Plants of Chile,accepted,subspecies,Hymenophyllum ferrugineum ssp. ferrugineum,Plantae,...,Polypodiopsida,Hymenophyllales,Hymenophyllaceae,Hymenophyllum,Hymenophyllum,None,ferrugineum,ferrugineum,None,ICBN
4,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20129,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20113,https://catalogoplantas.udec.cl/taxa/20113,Catalogue of Vascular Plants of Chile,accepted,subspecies,Koeleria spicata ssp. spicata,Plantae,...,Liliopsida,Poales,Poaceae,Koeleria,Koeleria,None,spicata,spicata,None,ICBN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1152,https://catalogoplantas.udec.cl/taxa/1406,https://catalogoplantas.udec.cl/taxa/239,https://catalogoplantas.udec.cl/taxa/1406,https://catalogoplantas.udec.cl/taxa/1406,https://catalogoplantas.udec.cl/taxa/1406,Catalogue of Vascular Plants of Chile,accepted,genus,Zephyra,Plantae,...,Liliopsida,Asparagales,Tecophilaeaceae,Zephyra,NaN,NaN,NaN,NaN,NaN,ICBN
1153,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/73,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/1407,https://catalogoplantas.udec.cl/taxa/1407,Catalogue of Vascular Plants of Chile,accepted,genus,Zephyranthes,Plantae,...,Liliopsida,Asparagales,Amaryllidaceae,Zephyranthes,NaN,NaN,NaN,NaN,NaN,ICBN
1154,https://catalogoplantas.udec.cl/taxa/1408,https://catalogoplantas.udec.cl/taxa/252,https://catalogoplantas.udec.cl/taxa/1408,https://catalogoplantas.udec.cl/taxa/1408,https://catalogoplantas.udec.cl/taxa/1408,Catalogue of Vascular Plants of Chile,accepted,genus,Zostera,Plantae,...,Liliopsida,Alismatales,Zosteraceae,Zostera,NaN,NaN,NaN,NaN,NaN,ICBN
1155,https://catalogoplantas.udec.cl/taxa/20091,https://catalogoplantas.udec.cl/taxa/203,https://catalogoplantas.udec.cl/taxa/20091,https://catalogoplantas.udec.cl/taxa/20091,https://catalogoplantas.udec.cl/taxa/20091,Catalogue of Vascular Plants of Chile,accepted,genus,Microlaena,Plantae,...,Liliopsida,Poales,Poaceae,Microlaena,NaN,NaN,NaN,NaN,NaN,ICBN


Add missing columns with `NULL` values

In [13]:
result["taxonRemarks"] = ""
result["nameAccordingTo"] = None
result["namePublishedIn"] = None
result["nomenclaturalStatus"] = None

In [14]:
catalogue.core = catalogue_core
catalogue.core.pandas = result
catalogue

Converting to pandas: 0entry [00:00, ?entry/s]


<Darwin Core Archive (Catalogue of Vascular Plants of Chile [Core: http://rs.tdwg.org/dwc/terms/Taxon, Entries: 20094])>

Save it

In [15]:
catalogue.to_file("data/vascular_plant_catalogue.zip")

Writing data http://rs.tdwg.org/dwc/terms/Taxon: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20094/20094 [00:01<00:00, 16939.18line/s]


<table>
    <tr>
        <td colspan="3" style="text-align: center;"><p>BIODATA - <a href="https://ieb-chile.cl/en/" target="_blank">Institute of Ecology and Biodiversity</a> © 2024</p></td>
    </tr>
</table>